In [1]:
import re
import tensorflow as tf
import numpy as np
import time
import os
import collections
import random
import time
import nltk
import pickle

In [2]:
def clearstring(string):
    string = re.sub('[^A-Za-z\- ]+', '', string)
    string = string.split(' ')
    string = filter(None, string)
    string = [y.strip() for y in string]
    return [y.lower() for y in string if len(y) > 3 and y.find('nbsp') < 0]

def build_dataset(words, n_words):
    count = [['UNK', -1]]
    count.extend(collections.Counter(words).most_common(n_words - 1))
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    data = list()
    unk_count = 0
    for word in words:
        index = dictionary.get(word, 0)
        if index == 0:  # dictionary['UNK']
            unk_count += 1
        data.append(index)
    count[0][1] = unk_count
    reversed_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data, count, dictionary, reversed_dictionary

data_index = 0

# Step 3: Function to generate a training batch for the skip-gram model.
def generate_batch(batch_size, num_skips, skip_window):
    global data_index
    global data
    assert batch_size % num_skips == 0
    assert num_skips <= 2 * skip_window
    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    span = 2 * skip_window + 1  # [ skip_window target skip_window ]
    buffer = collections.deque(maxlen=span)
    if data_index + span > len(data):
        data_index = 0
    buffer.extend(data[data_index:data_index + span])
    data_index += span
    for i in range(batch_size // num_skips):
        context_words = [w for w in range(span) if w != skip_window]
        words_to_use = random.sample(context_words, num_skips)
        for j, context_word in enumerate(words_to_use):
            batch[i * num_skips + j] = buffer[skip_window]
            labels[i * num_skips + j, 0] = buffer[context_word]
        if data_index == len(data):
            for word in data[:span]:
                buffer.append(word)
            data_index = span
        else:
            buffer.append(data[data_index])
            data_index += 1
    # Backtrack a little bit to avoid skipping words in the end of a batch
    data_index = (data_index + len(data) - span) % len(data)
    return batch, labels

In [3]:
with open('books/Azkaban.txt', 'r',encoding = "ISO-8859-1") as fopen:
    vocabulary = clearstring(' '.join(nltk.word_tokenize(fopen.read())))
print('example 10 words:',vocabulary[:10])
print('size corpus:',len(vocabulary))
vocabulary_size = len(list(set(vocabulary)))
print('size of unique words:',vocabulary_size)
dimension = 128
skip_window = 1
num_skips = 2
batch_size = 64
location = os.getcwd()

example 10 words: ['harry', 'potter', 'highly', 'unusual', 'many', 'ways', 'thing', 'hated', 'summer', 'holidays']
size corpus: 61523
size of unique words: 7181


In [4]:
data, count, dictionary, reverse_dictionary = build_dataset(vocabulary,vocabulary_size)
del vocabulary  # Hint to reduce memory.
print('Most common words (+UNK)', count[:5])
print('Sample data', data[:10], [reverse_dictionary[i] for i in data[:10]])

Most common words (+UNK) [['UNK', 1], ('harry', 2004), ('said', 1436), ('that', 862), ('they', 671)]
Sample data [1, 91, 963, 1381, 367, 1910, 134, 2585, 744, 714] ['harry', 'potter', 'highly', 'unusual', 'many', 'ways', 'thing', 'hated', 'summer', 'holidays']


In [5]:
valid_size = 16
valid_window = 100
valid_examples = np.random.choice(valid_window, valid_size, replace=False)

In [ ]:
graph = tf.Graph()

with graph.as_default():

    train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
    train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
    valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

    # Ops and variables pinned to the CPU because of missing GPU implementation
    with tf.device('/cpu:0'):
        embeddings = tf.Variable(tf.random_uniform([vocabulary_size, dimension], -1.0, 1.0))
        embed = tf.nn.embedding_lookup(embeddings, train_inputs)
        nce_weights = tf.Variable(tf.truncated_normal([vocabulary_size, dimension],
                                                      stddev=1.0 / np.sqrt(dimension)))
        nce_biases = tf.Variable(tf.zeros([vocabulary_size]))
        loss = tf.reduce_mean(tf.nn.nce_loss(weights=nce_weights,
                                             biases=nce_biases,
                                             labels=train_labels,
                                             inputs=embed,
                                             num_sampled=batch_size / 2,
                                             num_classes=vocabulary_size))
        optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(loss)

        # Compute the cosine similarity between minibatch examples and all embeddings.
        norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
        normalized_embeddings = embeddings / norm
        valid_embeddings = tf.nn.embedding_lookup(normalized_embeddings, valid_dataset)
        similarity = tf.matmul(valid_embeddings, normalized_embeddings, transpose_b=True)
        init = tf.global_variables_initializer()

num_steps = 200000

In [ ]:
with tf.Session(graph=graph) as session:
    init.run()
    print('Initialized')

    average_loss = 0
    for step in range(num_steps):
        batch_inputs, batch_labels = generate_batch(batch_size, num_skips, skip_window)
        feed_dict = {train_inputs: batch_inputs, train_labels: batch_labels}

        _, loss_val = session.run([optimizer, loss], feed_dict=feed_dict)
        average_loss += loss_val

        if step % 2000 == 0:
            if step > 0:
                average_loss /= 2000
            print('Average loss at step ', step, ': ', average_loss)
            average_loss = 0

        if step % 10000 == 0:
            sim = similarity.eval()
            for i in range(valid_size):
                valid_word = reverse_dictionary[valid_examples[i]]
                top_k = 8  # number of nearest neighbors
                nearest = (-sim[i, :]).argsort()[1:top_k + 1]
                log_str = 'Nearest to %s:' % valid_word
                for k in range(top_k):
                    close_word = reverse_dictionary[nearest[k]]
                    log_str = '%s %s,' % (log_str, close_word)
                print(log_str)
    final_embeddings = normalized_embeddings.eval()

Initialized
Average loss at step  0 :  104.917404175
Nearest to time: squish, allowed, tempted, expressionless, imps, misleading, count, firebold,
Nearest to pettigrew: deciding, grunting, bred, thoroughly, distractedly, fail, sloping, hardwearing,
Nearest to were: tone, secure, whooshed, only, frowning, bumps, earnestly, warning,
Nearest to though: promised, mixture, decorations, dint, rainfilled, bathed, scrawl, worsened,
Nearest to that: curling, lurked, among, noble, cream, armfuls, raptly, student,
Nearest to black: struck, alohomora, lightning, works, dissolved, yellowish, doggedly, security,
Nearest to UNK: folds, happens, glint, discovery, strapping, maliciously, pops, madman,
Nearest to very: uphold, redhaired, littered, suffer, spawn, quarter, reached, indicating,
Nearest to voice: mischiefmakers, whining, trembled, tiny, raised, farfetched, unless, hallucin,
Nearest to looking: fainted, aunts, rubbish, finished, beasts, instructing, thickened, teacher,
Nearest to dementors: 

Average loss at step  44000 :  2.64485992867
Average loss at step  46000 :  2.6228626616
Average loss at step  48000 :  2.60135991043
Average loss at step  50000 :  2.57015072471
Nearest to time: expected, nutter, trickling, peace, loathing, caterpillar, splashing, tampered,
Nearest to pettigrew: fail, reluctantly, deciding, dives, suit, omen, littered, dust,
Nearest to were: hardest, enraged, months, showers, circular, bumps, wendelin, flint,
Nearest to though: wish, donkey, counterclockwise, percy, brother, vernon, whenever, copied,
Nearest to that: eyebrow, solemnly, practices, emotions, unrestrainedly, surprised, lunge, explode,
Nearest to black: deciding, gloating, growl, security, assuming, clearwater, quarry, aware,
Nearest to UNK: said, happens, maliciously, glint, folds, madman, strapping, faults,
Nearest to very: indicating, stateoftheart, tramped, suffer, spawn, poised, expected, yelping,
Nearest to voice: yelping, hoods, applauded, davey, squealing, yours, threatened, under

Average loss at step  92000 :  2.30458558056
Average loss at step  94000 :  2.29141518134
Average loss at step  96000 :  2.29639391336
Average loss at step  98000 :  2.29550188524
Average loss at step  100000 :  2.27543947244
Nearest to time: loathing, jinxes, justice, unlocked, inherited, misleading, trickling, bogs,
Nearest to pettigrew: fail, reluctantly, cleansweeps, dives, littered, deciding, secondhand, apprehensively,
Nearest to were: enraged, been, coolly, showers, bumps, wendelin, hardest, wasn,
Nearest to though: pawed, alarming, doubled, brother, splat, donkey, drawing, nervouslooking,
Nearest to that: eyebrow, solemnly, practices, harry, lunge, unrestrainedly, comforted, heated,
Nearest to black: gloating, quarry, deciding, forbid, growl, remains, expertly, clearwater,
Nearest to UNK: said, split, tarnish, fail, maliciously, faults, madman, strapping,
Nearest to very: indicating, illassorted, suffer, tramped, yelping, blasting, poised, supplied,
Nearest to voice: yelping, s

Average loss at step  142000 :  2.22477472693
Average loss at step  144000 :  2.21648496422
Average loss at step  146000 :  2.22409052306


In [ ]:
with open('azkaban-list.p', 'wb') as fopen:
    pickle.dump(list(reverse_dictionary.values()), fopen)

In [ ]:
with open('azkaban-vector.p', 'wb') as fopen:
    pickle.dump(final_embeddings, fopen)